In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt


In [3]:
def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(indep_X,dep_Y)
        selectk_features = fit1.transform(indep_X)
        selected_columns = indep_X.columns[fit1.get_support(indices=True)]
        return selectk_features,selected_columns
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2


In [4]:

def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    
def svm_NL(X_train,y_train,X_test):       
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2    

def Decision(X_train,y_train,X_test):
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 

In [5]:
def selectk_regression(acclin, accsvml, accsvmnl, accdes, accrf): 
    dataframe = pd.DataFrame(index=['ChiSquare'], columns=['Linear', 'SVMl', 'SVMnl', 'Decision', 'Random'])
    
    for number, idex in enumerate(dataframe.index):
        dataframe.loc[idex, 'Linear'] = acclin[number]
        dataframe.loc[idex, 'SVMl'] = accsvml[number]
        dataframe.loc[idex, 'SVMnl'] = accsvmnl[number]
        dataframe.loc[idex, 'Decision'] = accdes[number]
        dataframe.loc[idex, 'Random'] = accrf[number]
    
    return dataframe

In [6]:
dataset1=pd.read_csv("prep.csv",index_col=None)

df2=dataset1

df2 = pd.get_dummies(df2,dtype=int,drop_first=True)

indep_x=df2.drop('classification_yes', axis=1)
dep_y=df2['classification_yes']

In [ ]:
df2

In [7]:
kbest,selected_columns =selectkbest(indep_x,dep_y,5)       

acclog=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

kbest

array([[1.48112676e+02, 5.74821053e+01, 3.07735602e+00, 3.88689024e+01,
        8.40819113e+03],
       [1.48112676e+02, 2.20000000e+01, 7.00000000e-01, 3.40000000e+01,
        1.23000000e+04],
       [9.90000000e+01, 2.30000000e+01, 6.00000000e-01, 3.40000000e+01,
        8.40819113e+03],
       ...,
       [1.10000000e+02, 1.15000000e+02, 6.00000000e+00, 2.60000000e+01,
        9.20000000e+03],
       [2.07000000e+02, 8.00000000e+01, 6.80000000e+00, 3.88689024e+01,
        8.40819113e+03],
       [1.00000000e+02, 4.90000000e+01, 1.00000000e+00, 5.30000000e+01,
        8.50000000e+03]])

In [8]:
X_train, X_test, y_train, y_test=split_scalar(kbest,dep_y)  
for i in kbest:   
    r2_lin=Linear(X_train,y_train,X_test)
    acclog.append(r2_lin)
    
    r2_sl=svm_linear(X_train,y_train,X_test)    
    accsvml.append(r2_sl)
    
    r2_NL=svm_NL(X_train,y_train,X_test)
    accsvmnl.append(r2_NL)
    
    r2_d=Decision(X_train,y_train,X_test)
    accdes.append(r2_d)
    
    r2_r=random(X_train,y_train,X_test)
    accrf.append(r2_r)
    
    
result=selectk_regression(acclog,accsvml,accsvmnl,accdes,accrf)


In [9]:
# Print the selected columns
print("Selected Columns for Feature Selection:", selected_columns)
result

Selected Columns for Feature Selection: Index(['bgr', 'bu', 'sc', 'pcv', 'wc'], dtype='object')


,Linear,SVMl,SVMnl,Decision,Random
ChiSquare,0.551985,0.545395,0.749654,0.696181,0.836806


In [12]:
# Subset the dataset with selected columns
X_selected = indep_x[selected_columns]

# Split the selected data into training and test sets
X_train_selected, X_test_selected, y_train, y_test = train_test_split(X_selected, dep_y, test_size=0.25, random_state=0)

# Scale the selected features
scaler = StandardScaler()
X_train_selected = scaler.fit_transform(X_train_selected)
X_test_selected = scaler.transform(X_test_selected)

# Train RandomForestRegressor using selected columns
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=10, random_state=0)
regressor.fit(X_train_selected, y_train)
r2=r2_prediction(regressor,X_test_selected,y_test)

print("r2_score:\n", r2)


r2_score:
 0.8368055555555556
